In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# Get 2013 middle school MCAS performance
# Performance is the average advanced or proficient % per grade weighted by the number of students in the grade

dfs = []
for file in os.listdir('xlsx'):
    if 'mcas_2013' in file:
        df = pd.read_excel(f'xlsx/{file}', header=1, dtype=str).astype(str)
        df = df[df['Subject'] == 'MATHEMATICS']
        df = df.rename(columns={'School Code':'school_code'})
        df = df.set_index('school_code')
        df = df[['P+A %', 'No. of Students Included']]
        df = df.astype(float)
        dfs.append(df)
df = pd.concat(dfs, axis=0)
df = df.rename(columns={'P+A %': 'x', 'No. of Students Included': 'w'})
df['xw'] = df['x'] * df['w']
group = df.groupby('school_code')
performance = group['xw'].sum() / group['w'].sum()
performance = performance.rename('performance')
performance = performance.reset_index()
performance = performance.set_index('school_code')
performance = performance[performance.index != '00000000']

print('Get average performance for each middle school')
print(performance.shape)

print('Save just the performance')
performance.to_csv('csv/performance.csv')

Get average performance for each middle school
(632, 1)
Save just the performance


In [3]:
# Combine all the demographics for the 2013 school year

demographics = pd.DataFrame(index=performance.index)
for file in os.listdir('xlsx'):
    if '2013' in file and 'mcas' not in file:
        df = pd.read_excel(f'xlsx/{file}', header=1, dtype=str).astype(str)
        df = df.iloc[:, 1:]
        df = df.rename(columns={df.columns[0]: 'school_code'})
        df = df.set_index('school_code')
        df = df.applymap(lambda x: x.replace(' ', '').replace(',', '').replace('to1', '').replace('######', 'NaN'))
        df = df.astype(float)
        df = df[[c for c in df.columns if c not in demographics.columns]]
        demographics = demographics.merge(df, how='outer', left_index=True, right_index=True)

print('Label all charter schools')
active_charter_schools = pd.read_excel(f'xlsx/charter-schools.xlsx', header=0, dtype=str).astype(str)
school_names = pd.read_excel(f'xlsx/educator-age_2013.xlsx', header=1, dtype=str).astype(str)
school_names['School Name'] = school_names['School Name'].str.lower()
school_names = school_names[school_names['School Name'].str.contains('innovation') | school_names['School Name'].str.contains('charter')]
charter_schools = np.unique(active_charter_schools['Org Code'].tolist() + school_names['School Code'].tolist())
demographics['charter_school'] = demographics.index.isin(charter_schools).astype(int)
print(demographics['charter_school'].sum())

print('Remove demographics from non middle schools')
demographics = demographics[demographics.index.isin(performance.index)]
print(demographics.shape)

print('Remove unchanging demographics')
demographics = demographics[demographics.columns[demographics.nunique(dropna=False) > 1]]
print(demographics.shape)

print('Remove perfectly correlated demographics')
columns_to_remove = []
corr_matrix = demographics.corr()
for a in demographics.columns:
    for b in demographics.columns:
        if a not in columns_to_remove and b not in columns_to_remove and a != b and np.abs(corr_matrix.loc[a, b]) == 1:
            columns_to_remove.append(b)
demographics = demographics.drop(columns=columns_to_remove)
print(demographics.shape)

print('Create dataframe of whether or not the other dataframe is nan')
nan_demographics = demographics.isna().astype(int)
nan_demographics.columns = [f'{c}_is_nan' for c in nan_demographics.columns]
print(nan_demographics.shape)

print('Remove unchanging nan demographics')
nan_demographics = nan_demographics[nan_demographics.columns[nan_demographics.nunique(dropna=False) > 1]]
print(nan_demographics.shape)

print('Remove perfectly correlated nan demographics')
columns_to_remove = []
corr_matrix = nan_demographics.corr()
for a in nan_demographics.columns:
    for b in nan_demographics.columns:
        if a not in columns_to_remove and b not in columns_to_remove and a != b and np.abs(corr_matrix.loc[a, b]) == 1:
            columns_to_remove.append(b)
nan_demographics = nan_demographics.drop(columns=columns_to_remove)
print(nan_demographics.shape)

print('Combine demographics with nan demographics')
demographics = pd.concat([demographics, nan_demographics], axis=1)
print(demographics.shape)

print('Save just the demogrpahics')
demographics.to_csv('csv/demographics.csv')

Label all charter schools
81
Remove demographics from non middle schools
(632, 69)
Remove unchanging demographics
(632, 69)
Remove perfectly correlated demographics
(632, 69)
Create dataframe of whether or not the other dataframe is nan
(632, 69)
Remove unchanging nan demographics
(632, 6)
Remove perfectly correlated nan demographics
(632, 1)
Combine demographics with nan demographics
(632, 70)
Save just the demogrpahics


In [4]:
# Get prior middle school MCAS results

priors = []
for year in [2007, 2008, 2009, 2010, 2011]:
    year_df = pd.DataFrame(index=performance.index)
    year_df['year'] = year
    for grade in [6, 7, 8]:
        grade_df = pd.read_excel(f'xlsx/mcas_{year}-{grade}.xlsx', header=1, dtype=str).astype(str)
        grade_df = grade_df[grade_df['Subject'] == 'MATHEMATICS']
        grade_df = grade_df.drop(columns=['School Name', 'Subject'])
        grade_df = grade_df.rename(columns={'School Code':'school_code'})
        grade_df = grade_df.set_index('school_code')
        grade_df = grade_df.applymap(lambda x: x.replace('nan', ''))
        grade_df = grade_df.apply(lambda x: pd.to_numeric(x.str.strip()))
        grade_df = grade_df.astype(float)
        grade_df.columns = [f'Grade {grade} {c}' for c in grade_df.columns]
        year_df = year_df.merge(grade_df, how='outer', left_index=True, right_index=True)
    priors.append(year_df)

raw_priors = pd.concat(priors, axis=0)

print('Remove priors from non middle schools')
raw_priors = raw_priors[raw_priors.index.isin(performance.index)]
print(raw_priors.shape)

priors = raw_priors.copy()

print('Remove unchanging priors')
priors = priors[priors.columns[priors.nunique(dropna=False) > 1]]
print(priors.shape)

print('Remove perfectly correlated priors')
columns_to_remove = []
corr_matrix = priors.corr()
for a in priors.columns:
    for b in priors.columns:
        if a not in columns_to_remove and b not in columns_to_remove and a != b and np.abs(corr_matrix.loc[a, b]) == 1:
            columns_to_remove.append(b)
priors = priors.drop(columns=columns_to_remove)
print(priors.shape)

print('Create dataframe of whether or not the other dataframe is nan')
nan_priors = priors.isna().astype(int)
nan_priors.columns = [f'{c}_is_nan' for c in nan_priors.columns]
print(nan_priors.shape)

print('Remove unchanging nan priors')
nan_priors = nan_priors[nan_priors.columns[nan_priors.nunique(dropna=False) > 1]]
print(nan_priors.shape)

print('Remove perfectly correlated nan priors')
columns_to_remove = []
corr_matrix = nan_priors.corr()
for a in nan_priors.columns:
    for b in nan_priors.columns:
        if a not in columns_to_remove and b not in columns_to_remove and a != b and np.abs(corr_matrix.loc[a, b]) == 1:
            columns_to_remove.append(b)
nan_priors = nan_priors.drop(columns=columns_to_remove)
print(nan_priors.shape)

print('Combine priors with nan priors')
priors = pd.concat([priors, nan_priors], axis=1)
print(priors.shape)

print('Save just the priors')
priors.to_csv('csv/prior_performance.csv')

print('Reshape priors into series of years')
stacked_priors = priors.set_index('year', append=True)
stacked_priors = stacked_priors.groupby('school_code').apply(lambda x: x.sort_values('year').values.tolist())
stacked_priors = stacked_priors.rename('prior_performance')
print(stacked_priors.shape)

Remove priors from non middle schools
(3160, 43)
Remove unchanging priors
(3160, 43)
Remove perfectly correlated priors
(3160, 43)
Create dataframe of whether or not the other dataframe is nan
(3160, 43)
Remove unchanging nan priors
(3160, 42)
Remove perfectly correlated nan priors
(3160, 9)
Combine priors with nan priors
(3160, 52)
Save just the priors
Reshape priors into series of years
(632,)


In [5]:
def flatten_priors(df):
    rows = []
    for _, row in df.iterrows():
        year = str(row['year']) + '_'
        new_row = row.drop(labels='year')
        new_row.index = year + new_row.index
        rows.append(new_row)
    flat_row = pd.concat(rows, axis=0)
    return flat_row

print('Flatten priors')
priors = raw_priors.groupby('school_code').apply(lambda x: flatten_priors(x))
print(priors.shape)

print('Merge demographics and priors')
df = demographics.merge(priors, how='inner', left_index=True, right_index=True)
print(df.shape)

print('Remove unchanging priors')
df = df[df.columns[df.nunique(dropna=False) > 1]]
print(df.shape)

print('Remove perfectly correlated priors')
columns_to_remove = []
corr_matrix = df.corr()
for a in df.columns:
    for b in df.columns:
        if a not in columns_to_remove and b not in columns_to_remove and a != b and np.abs(corr_matrix.loc[a, b]) == 1:
            columns_to_remove.append(b)
df = df.drop(columns=columns_to_remove)
print(df.shape)

print('Create dataframe of whether or not the other dataframe is nan')
nan_df = df.isna().astype(int)
nan_df.columns = [f'{c}_is_nan' for c in nan_df.columns]
print(nan_df.shape)

print('Remove unchanging nan priors')
nan_df = nan_df[nan_df.columns[nan_df.nunique(dropna=False) > 1]]
print(nan_df.shape)

print('Remove perfectly correlated nan priors')
columns_to_remove = []
corr_matrix = nan_df.corr()
for a in nan_df.columns:
    for b in nan_df.columns:
        if a not in columns_to_remove and b not in columns_to_remove and a != b and np.abs(corr_matrix.loc[a, b]) == 1:
            columns_to_remove.append(b)
nan_df = nan_df.drop(columns=columns_to_remove)
print(df.shape)

print('Combine priors with nan priors')
df = pd.concat([df, nan_df], axis=1)
print(df.shape)

print('Save flattened data')
df.to_csv('csv/flat_mass_doe_data.csv')

Flatten priors
(632, 210)
Merge demographics and priors
(632, 280)
Remove unchanging priors
(632, 274)
Remove perfectly correlated priors
(632, 274)
Create dataframe of whether or not the other dataframe is nan
(632, 274)
Remove unchanging nan priors
(632, 210)
Remove perfectly correlated nan priors
(632, 274)
Combine priors with nan priors
(632, 303)
Save flattened data


In [6]:
df = demographics.merge(stacked_priors, how='inner', left_index=True, right_index=True)
df.to_csv('csv/mass_doe_data.csv')